In [1]:
import requests
import os
import re
import sys
import json
import requests
import argparse
import time
import codecs
from bs4 import BeautifulSoup
from six import u

In [2]:
import requests

def fetch(url):
    response = requests.get(url)
    response = requests.get(url, cookies={'over18': '1'})  # 一直向 server 回答滿 18 歲了 !
    return response

url = 'https://www.ptt.cc/bbs/movie/index.html'
resp = fetch(url)  # step-1

print(resp.text) # result of setp-1

<!DOCTYPE html>
<html>
	<head>
		<meta charset="utf-8">
		

<meta name="viewport" content="width=device-width, initial-scale=1">

<title>看板 movie 文章列表 - 批踢踢實業坊</title>

<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.26/bbs-common.css">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.26/bbs-base.css" media="screen">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.26/bbs-custom.css">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.26/pushstream.css" media="screen">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.26/bbs-print.css" media="print">




	</head>
    <body>
		
<div id="topbar-container">
	<div id="topbar" class="bbs-content">
		<a id="logo" href="/bbs/">批踢踢實業坊</a>
		<span>&rsaquo;</span>
		<a class="board" href="/bbs/movie/index.html"><span class="board-label">看板 </span>movie</a>
		<a class="right small" href="/about.html">關於我們</a>
		<a class="right small" href="/contact.html">

In [3]:
from requests_html import HTML

def parse_article_entries(doc):
    html = HTML(html=doc)
    post_entries = html.find('div.r-ent')
    return post_entries

url = 'https://www.ptt.cc/bbs/movie/index.html'
resp = fetch(url)  # step-1
post_entries = parse_article_entries(resp.text)  # step-2
print(post_entries)  # result of setp-2

[<Element 'div' class=('r-ent',)>, <Element 'div' class=('r-ent',)>, <Element 'div' class=('r-ent',)>, <Element 'div' class=('r-ent',)>, <Element 'div' class=('r-ent',)>, <Element 'div' class=('r-ent',)>, <Element 'div' class=('r-ent',)>, <Element 'div' class=('r-ent',)>, <Element 'div' class=('r-ent',)>, <Element 'div' class=('r-ent',)>, <Element 'div' class=('r-ent',)>, <Element 'div' class=('r-ent',)>, <Element 'div' class=('r-ent',)>, <Element 'div' class=('r-ent',)>, <Element 'div' class=('r-ent',)>, <Element 'div' class=('r-ent',)>, <Element 'div' class=('r-ent',)>, <Element 'div' class=('r-ent',)>, <Element 'div' class=('r-ent',)>, <Element 'div' class=('r-ent',)>, <Element 'div' class=('r-ent',)>, <Element 'div' class=('r-ent',)>]


In [4]:
def parse_article_meta(entry):
    '''
    每筆資料都存在 dict() 類型中：key-value paird data
    '''
    return {
        'title': entry.find('div.title', first=True).text,
        'push': entry.find('div.nrec', first=True).text,
        'date': entry.find('div.date', first=True).text,
        'author': entry.find('div.author', first=True).text,
        'link': entry.find('div.title > a', first=True).attrs['href'],
    }

url = 'https://www.ptt.cc/bbs/movie/index.html'
resp = fetch(url)  # step-1
post_entries = parse_article_entries(resp.text)  # step-2

for entry in post_entries:
    meta = parse_article_meta(entry)
    print(meta)  # result of setp-3

    # pretty_print(meta['push'], meta['title'], meta['date'], meta['author'])  # for below results

{'title': '[討論] 李安新作【雙子殺手】另一版最新預告', 'push': '12', 'date': '7/26', 'author': 'arsl400', 'link': '/bbs/movie/M.1564072187.A.7BF.html'}
{'title': '[討論] 寄生上流讓我出戲的一幕 （有雷）', 'push': '1', 'date': '7/26', 'author': 'jack281600', 'link': '/bbs/movie/M.1564073592.A.EF5.html'}


AttributeError: 'NoneType' object has no attribute 'attrs'

In [5]:
widths = [
        (126,    1), (159,    0), (687,     1), (710,   0), (711,   1),
        (727,    0), (733,    1), (879,     0), (1154,  1), (1161,  0),
        (4347,   1), (4447,   2), (7467,    1), (7521,  0), (8369,  1),
        (8426,   0), (9000,   1), (9002,    2), (11021, 1), (12350, 2),
        (12351,  1), (12438,  2), (12442,   0), (19893, 2), (19967, 1),
        (55203,  2), (63743,  1), (64106,   2), (65039, 1), (65059, 0),
        (65131,  2), (65279,  1), (65376,   2), (65500, 1), (65510, 2),
        (120831, 1), (262141, 2), (1114109, 1),
]


def calc_len(string):
    def chr_width(o):
        global widths
        if o == 0xe or o == 0xf:
            return 0
        for num, wid in widths:
            if o <= num:
                return wid
        return 1
    return sum(chr_width(ord(c)) for c in string)


def pretty_print(push, title, date, author):
    pattern = '%3s\t%s%s%s\t%s'
    padding = ' ' * (50 - calc_len(title))
    print(pattern % (push, title, padding, date, author))

In [6]:
controls = html.find('.action-bar a.btn.wide')

NameError: name 'html' is not defined

In [7]:
link = controls[1].get('href')
page_url = urllib.parse.urljoin('https://www.ptt.cc/', link)

NameError: name 'controls' is not defined

In [11]:
def parse_article_meta(ent):
    ''' Step-3 (revised): parse the metadata in article entry '''
    # 基本要素都還在
    meta = {
        'title': ent.find('div.title', first=True).text,
        'push': ent.find('div.nrec', first=True).text,
        'date': ent.find('div.date', first=True).text,
    }

    try:
        # 正常狀況取得資料
        meta['author'] = ent.find('div.author', first=True).text
        meta['link'] = ent.find('div.title > a', first=True).attrs['href']
    except AttributeError:
        # 但碰上文章被刪除時，就沒有辦法像原本的方法取得 作者 跟 連結
        if '(本文已被刪除)' in meta['title']:
            # e.g., "(本文已被刪除) [haudai]"
            match_author = re.search('\[(\w*)\]', meta['title'])
            if match_author:
                meta['author'] = match_author.group(1)
        elif re.search('已被\w*刪除', meta['title']):
            # e.g., "(已被cappa刪除) <edisonchu> op"
            match_author = re.search('\<(\w*)\>', meta['title'])
            if match_author:
                meta['author'] = match_author.group(1)
    return meta

    # 最終仍回傳統一的 dict() 形式 paired data
    return meta

In [32]:
import urllib.parse
from imp import reload
import utils
reload(utils)
from utils import pretty_print

def get_metadata_from(url):

    def parse_next_link(doc):
        html = HTML(html=doc)
        controls = html.find('.action-bar a.btn.wide')
        link = controls[1].attrs.get('href')
        return urllib.parse.urljoin('https://www.ptt.cc/', link)

    resp = fetch(url)
    post_entries = parse_article_entries(resp.text)
    next_link = parse_next_link(resp.text)

    metadata = [parse_article_meta(entry) for entry in post_entries]
    return metadata, next_link

def get_paged_meta(url, num_pages):
    collected_meta = []

    for _ in range(num_pages):
        posts, link = get_metadata_from(url)
        collected_meta += posts
        url = urllib.parse.urljoin('https://www.ptt.cc/', link)

    return collected_meta

start_url = 'https://www.ptt.cc/bbs/movie/index.html'
metadata = get_paged_meta(start_url, num_pages=3)
for meta in metadata:
    pretty_print(meta['push'], meta['title'], meta['date'], meta['author'])

 12	[討論] 李安新作【雙子殺手】另一版最新預告         	7/26	arsl400
  1	[討論] 寄生上流讓我出戲的一幕 （有雷）            	7/26	jack281600
   	(本文已被刪除) [arsl400]                          	7/26	arsl400
 66	[負雷] 超智障環太平洋                             	7/26	Dissipate
   	[好雷] 首席指揮家 - 群「雄」環視的窄門與高牆      	7/26	sora7936
  4	[問片] 一部美國西部片（有雷）                     	7/26	popher
 14	[請益] 普羅米修斯                                 	7/26	dickygto
 74	[好雷]從前有個好萊塢，一些補充                    	7/26	LN0222
  8	[請益] 灼人秘密的幾個問題 有雷                    	7/26	poundingface
  1	Re: [請益] 秦頌的幾個疑問 (微雷)                  	7/26	monarch0301
 21	[片單] 有年齡差異大的手足的片                     	7/26	yuxxoholic
  3	[情報] itunes 一級玩家 150元                      	7/26	Angesi
  6	Re: [討論] 【雙子殺手】最新預告                   	7/26	saiiys
 11	Re: [好雷]從前有個好萊塢，一些補充                	7/26	shyuwu
  3	[請益] 港片結尾演員名單                           	7/26	cooper0710
  1	[討論] 電流大戰                                   	7/26	b13b46o2
   	(本文已被刪除) [zzahoward]                        	7/26	zzahoward
 36	[片單] 請問能推薦療癒的電影嗎

In [34]:
print(0xf)

15
